In [1]:
!pip install datasets
!pip install loralib
!pip install trl
!pip install accelerate
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 72.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 20.0.0
    Uninstalling pyarrow-20.0.0:
      Successfully uninstalled pyarrow-20.0.0━━━━━━━━━━━━━━━━━━━━━  1/13 [pyarrow]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [datasets]/13 [datasets]ce-hub]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.0/802.0 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [trl]5/6 [trl]sformers]


In [2]:
!git clone https://github.com/airobotlab/KoChatGPT
!cp -r KoChatGPT/colossalai_ChatGPT_230319/chatgpt chatgpt

Cloning into 'KoChatGPT'...
remote: Enumerating objects: 304, done.
remote: Total 304 (delta 0), reused 0 (delta 0), pack-reused 304 (from 1)
Receiving objects: 100% (304/304), 57.72 MiB | 19.42 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [3]:
import os

modifications = [
    {
        "file": "chatgpt/trainer/callbacks/save_checkpoint.py",
        "changes": [
            {"line": 3, "old": "from chatgpt.trainer.strategies import ColossalAIStrategy, Strategy",
             "new": "from chatgpt.trainer.strategies import Strategy"},
            {"line": 71, "old": "only_rank0 = not isinstance(self.strategy, ColossalAIStrategy)",
             "new": "            only_rank0 = not isinstance(self.strategy)"},
        ],
    },
    {
        "file": "chatgpt/trainer/strategies/__init__.py",
        "changes": [
            {"line": 1, "old": "from .colossalai import ColossalAIStrategy", "new": ""},  # 삭제
            {"line": 5, "old": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy', 'ColossalAIStrategy']",
             "new": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy']"},
        ],
    },
    {
        "file": "chatgpt/dataset/reward_dataset.py",
        "changes": [
            {"line": 3, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ],
    },
    {
        "file": "chatgpt/trainer/strategies/__init__.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    },
    {
        "file": "chatgpt/dataset/reward_dataset.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    }
]


def modify_file(file_path, changes):
    """파일에서 지정된 줄을 찾아 내용을 수정하는 함수"""

    if not os.path.exists(file_path):
        print(f"⚠️ 파일이 존재하지 않습니다: {file_path}")
        return

    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    modified = False

    for change in changes:
        line_index = change["line"]
        if 0 <= line_index < len(lines):
            if lines[line_index].strip() == change["old"]:
                lines[line_index] = change["new"] + "\n"
                modified = True
            else:
                print(f"⚠️ {file_path} 파일의 {change['line']}번째 줄이 예상과 다릅니다.")
                print(f"   예상: {change['old']}")
                print(f"   실제: {lines[line_index].strip()}")

    if modified:
        with open(file_path, "w", encoding="utf-8") as file:
            file.writelines(lines)
        print(f"✅ 수정 완료: {file_path}")
    else:
        print(f"⚠️ {file_path} 수정할 내용이 없습니다.")

for mod in modifications:
    modify_file(mod["file"], mod["changes"])

✅ 수정 완료: chatgpt/trainer/callbacks/save_checkpoint.py
✅ 수정 완료: chatgpt/trainer/strategies/__init__.py
✅ 수정 완료: chatgpt/dataset/reward_dataset.py
⚠️ chatgpt/trainer/strategies/__init__.py 수정할 내용이 없습니다.
⚠️ chatgpt/dataset/reward_dataset.py 파일의 8번째 줄이 예상과 다릅니다.
   예상: from tqdm import tqdm
   실제: class RewardDataset(Dataset):
⚠️ chatgpt/dataset/reward_dataset.py 수정할 내용이 없습니다.


In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
print("transformers version: {}".format(transformers.__version__)) # transformers 4.28.0
print("GPU 사용 가능여부: {}".format(torch.cuda.is_available()))

# 만일 아래 모듈이 불러와지지 않는다면 Clone 및 수정을 잘 진행했는지 확인해주세요.
from chatgpt.trainer.strategies import NaiveStrategy

Torch version:2.7.1+cu118
Cuda version: 11.8
transformers version: 4.56.1
GPU 사용 가능여부: True


In [34]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [35]:
from typing import Optional, Dict, Sequence
from torch.utils.data import Dataset
from dataclasses import dataclass
import logging
import copy

In [36]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<usr>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<sys>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=

In [37]:

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [38]:
@dataclass
class DataCollatorForSupervisedDataset(object):

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [39]:
import json

In [40]:
train_dataset = SFT_dataset(data_path_1_SFT='KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [41]:
training_args = transformers.TrainingArguments(
    output_dir="test",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=5,
    report_to="none",
    prediction_loss_only=True,
    fp16 = True
    )
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [42]:
trainer.train()
model.save_pretrained('models/output_1_SFT')

Step,Training Loss
500,2.975400
1000,2.781900
1500,2.684700


In [43]:
generator = transformers.pipeline('text-generation', model='models/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

Device set to use cuda:0



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'죄송합니다, 저는 인공지능 언어모델로써 답변을 생성하는 AI 어시스턴트이기 때문에 정확한 답변을 제공할 수 없습니다. 하지만 일반적으로 불고기용 고기는 쇠고기의 종류에 따라 다양한 종류가 있을 수 있습니다. 예를 들어, 소고기, 돼지고기, 닭고기 등 다양한 종류의 불고기용

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 41대 부통령직을 수행했습니다.son은 47대 부통령직을 역임하였습니다.son이 46대 부통령직을 맡은 년도는 1946년입니다.son은 1949년에 부통령직을 맡았습니다.son의 부통령직 수행년도는 1952년입니다.son과 그의 부통령직 수행년에 대한 자세한 정보는

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 시카고에 대한 정보를 가지고 있지 않습니다. 하지만 시카고는 미국 캘리포니아주 로스앤젤레스에 위치한 도시입니다. 시카고는 미국에서 가장 큰 도시 중 하나이며, 많은 사람들이 살고 있는 지역 중 하나입니다. 따라서 시카고는 미국 내에서 가장 인기 있는 도시 중 하나입니다.子, "시카고

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 미세먼지에 대한 정보를 알 수 없습니다. 하지만 미세먼지 예보 앱을 통해 미세먼지 농도를 확인해보시는 것을 추천드립니다. 또한 미세먼지 농도가 높은 날에는 대중교통을 이용하시는 것도 좋은 방법입니다. 만약 미세먼지가 심한


In [44]:
model.save_pretrained('models/output_1_SFT')

In [17]:
torch.cuda.empty_cache()

In [19]:
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer.strategies import NaiveStrategy
from chatgpt.trainer.rm import RewardModelTrainer

from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

import torch.nn as nn

import random

In [47]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [48]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

In [49]:
with open('KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [50]:
import random
random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은?', 'chosen': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은 류승완의 사무실입니다.', 'rejected': '대구 영화사옥'}


In [51]:
train_data = total_data_ranking2chosen[:1000]
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [52]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=torch.optim.Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

In [53]:
trainer.fit(use_lora=0)



Train epoch:   0%|          | 0/1 [00:00<?, ?it/s]


Train step of epoch 0:   0%|          | 0/250 [00:00<?, ?it/s]


Train step of epoch 0:   0%|          | 1/250 [00:00<02:45,  1.50it/s]


Train step of epoch 0:   0%|          | 1/250 [00:00<02:45,  1.50it/s, loss=0.725]


Train step of epoch 0:   1%|          | 2/250 [00:01<03:00,  1.37it/s, loss=0.725]


Train step of epoch 0:   1%|          | 2/250 [00:01<03:00,  1.37it/s, loss=0.881]


Train step of epoch 0:   1%|          | 3/250 [00:02<03:18,  1.24it/s, loss=0.881]


Train step of epoch 0:   1%|          | 3/250 [00:02<03:18,  1.24it/s, loss=0.56] 


Train step of epoch 0:   2%|▏         | 4/250 [00:03<03:26,  1.19it/s, loss=0.56]


Train step of epoch 0:   2%|▏         | 4/250 [00:03<03:26,  1.19it/s, loss=0.534]


Train step of epoch 0:   2%|▏         | 5/250 [00:04<03:30,  1.16it/s, loss=0.534]


Train step of epoch 0:   2%|▏         | 5/250 [00:04<03:30,  1.16it/s, loss=0.446]


Train step of epoch 0:   2%|▏         | 6/2

In [55]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').cuda()
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: -0.7


In [32]:
model.save_pretrained('models/output_2_RM')

In [56]:
my_candidates = [ 
    "서울 어린이대공원 동물원이나 놀이시설도 추천해요. 입장료가 저렴해서 가족 단위 방문객이 많습니다.",
    "날씨가 좋으면 잠실 롯데월드타워 전망대, 실내활동이면 국립중앙박물관 어린이박물관도 좋아요.",
    "아이 연령대에 따라 다르지만, 5~7세라면 코엑스 키자니아 같은 체험형 공간이 적합합니다.",

    "서울에는 다양한 체험 공간이 있어요. 인터넷 검색해보시면 많은 정보를 얻을 수 있습니다.",
    "추천드릴 수는 없지만 서울에는 어린이들을 위한 좋은 장소가 많습니다.",
    "아이와 함께라면 실내 전시관이나 놀이시설이 무난합니다.",

    "저는 인공지능이기 때문에 추천이 불가능합니다.",
    "알려드릴 수 없습니다.",
    "모릅니다."
]


In [57]:
for i in my_candidates:
    inference_RM(i)

input: 서울 어린이대공원 동물원이나 놀이시설도 추천해요. 입장료가 저렴해서 가족 단위 방문객이 많습니다.
reward score: -0.6
input: 날씨가 좋으면 잠실 롯데월드타워 전망대, 실내활동이면 국립중앙박물관 어린이박물관도 좋아요.
reward score: -0.9
input: 아이 연령대에 따라 다르지만, 5~7세라면 코엑스 키자니아 같은 체험형 공간이 적합합니다.
reward score: -0.7
input: 서울에는 다양한 체험 공간이 있어요. 인터넷 검색해보시면 많은 정보를 얻을 수 있습니다.
reward score: -0.9
input: 추천드릴 수는 없지만 서울에는 어린이들을 위한 좋은 장소가 많습니다.
reward score: -0.8
input: 아이와 함께라면 실내 전시관이나 놀이시설이 무난합니다.
reward score: -1.0
input: 저는 인공지능이기 때문에 추천이 불가능합니다.
reward score: -0.9
input: 알려드릴 수 없습니다.
reward score: -1.1
input: 모릅니다.
reward score: -3.4


In [58]:
health_candidates = [
    "규칙적인 운동과 충분한 수면이 가장 중요합니다. 하루 30분 정도 빠르게 걷기만 해도 건강에 큰 도움이 됩니다.",
    "채소와 과일을 자주 섭취하고, 가공식품과 설탕 섭취를 줄이세요. 또한 하루에 물을 1.5리터 이상 마시는 것이 좋습니다.",
    "스트레칭은 간단하지만 효과적이에요. 아침에 5분만 해도 혈액순환과 집중력 향상에 도움이 됩니다.",


    "건강을 지키려면 꾸준히 관리하는 것이 필요합니다.",
    "운동도 좋고 식습관도 중요합니다.",
    "잘 먹고 잘 자는 것이 건강에 좋아요.",

    "저는 인공지능이기 때문에 건강 관리 방법을 알려드릴 수 없습니다.",
    "모릅니다.",
    "추천할 수 없습니다."
]


In [59]:
for i in my_candidates:
    inference_RM(i)

input: 서울 어린이대공원 동물원이나 놀이시설도 추천해요. 입장료가 저렴해서 가족 단위 방문객이 많습니다.
reward score: -0.6
input: 날씨가 좋으면 잠실 롯데월드타워 전망대, 실내활동이면 국립중앙박물관 어린이박물관도 좋아요.
reward score: -0.9
input: 아이 연령대에 따라 다르지만, 5~7세라면 코엑스 키자니아 같은 체험형 공간이 적합합니다.
reward score: -0.7
input: 서울에는 다양한 체험 공간이 있어요. 인터넷 검색해보시면 많은 정보를 얻을 수 있습니다.
reward score: -0.9
input: 추천드릴 수는 없지만 서울에는 어린이들을 위한 좋은 장소가 많습니다.
reward score: -0.8
input: 아이와 함께라면 실내 전시관이나 놀이시설이 무난합니다.
reward score: -1.0
input: 저는 인공지능이기 때문에 추천이 불가능합니다.
reward score: -0.9
input: 알려드릴 수 없습니다.
reward score: -1.1
input: 모릅니다.
reward score: -3.4


In [60]:
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer

from copy import deepcopy

In [61]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='models/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='models/output_2_RM', lora_rank=0).to(torch.cuda.current_device())
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right",
        model_max_length=512
    )
    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [62]:
actor_optim = torch.optim.Adam(actor.parameters(), lr=5e-6)
critic_optim = torch.optim.Adam(critic.parameters(), lr=5e-6)

In [63]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [64]:
with open('KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [65]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,
                     train_batch_size=8,
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [66]:
trainer.fit(list_prompt,
            num_episodes=10,
            max_timesteps=3,
            update_timesteps=3)

actor.model.save_pretrained('models/output_3_PPO')



Episode [1/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Episode [1/10]:  33%|███▎      | 1/3 [00:06<00:12,  6.35s/it]

Episode [1/10]:  67%|██████▋   | 2/3 [00:12<00:06,  6.45s/it]


Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s]


Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s, actor_loss=0, critic_loss=0.000796]


Train epoch [1/1]:  33%|███▎      | 1/3 [00:00<00:01,  1.85it/s, actor_loss=0, critic_loss=0.000796]


Train epoch [1/1]:  33%|███▎      | 1/3 [00:01<00:01,  1.85it/s, actor_loss=0, critic_loss=0.0802]  


Train epoch [1/1]:  67%|██████▋   | 2/3 [00:01<00:00,  1.87it/s, actor_loss=0, critic_loss=0.0802]


Train epoch [1/1]:  67%|██████▋   | 2/3 [00:01<00:00,  1.87it/s, actor_loss=0, critic_loss=0.00772]


Train epoch [1/1]: 100%|██████████| 3/3 [00:01<00:00,  1.88it/s, actor_loss=0, critic_loss=0.00772]


Episode [1/10]: 100%|██████████| 3/3 [00:20<00:00,  6.96s/it]


Episode [2/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Episode [2/10]:  33%|███▎      |

In [67]:
def generation(input_text, model):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = model.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?',
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text, actor)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'아니다. 일반적으로 고기를 살 때 보통 맵고 부드러운 고기와 고기를 사용하는 것이 권장됩니다. 그러나 고기를 조리하는 방식은 다양하므로 다양한 방법으로 선택할 수 있습니다. 따라서 고기를 조리할 때는 정확한 맛과 식량, 조리 방법에 대해 잘 파악하시고 선택하시는 것이 좋습니다.哉), 불고기용 고기 한우는 주로 고기, 쇠고기, 양고기, 불고기, 닭볶음탕 등이 제공됩니다.哉), 불고기용 고기를 사용하는 이유는 일반적인 상황에서 사용되는 고기 고기보다 더 부드럽고 부드러운 맛과 함께 고기가 제공된다는 장점 때문입니다.哉), 불고기용 고기 한우는 매운맛과 매운맛을 동시에 지니고 있어 매운 맛을 강조할 수 있고, 다양한 음식 재료로 활용할 수 있다는 장점이 있습니다.哉), 불고기용 고기 한우는 일반적으로 소스, 국산, 불고기 등으로 제공됩니다.哉), 불고기용 한우는 주로 소스, 국산, 소스, 혼합, 혼합, 육류, 고기, 불고기용으로 제공됩니다.哉)는 고기를 조리할 때 사용되는 재료와 종류의 혼합과 섞어 요리합니다.哉)는 고기를 조리할 때 사용되는 재료와 형태,

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨이 47대 부통령직을 수행한 년도는 1948년입니다. Johnson Conner이 맡았던 년도입니다. Johnson은 1972년에 부통령직에 임명되었습니다. Johnson은 1978년에 부통령직에 임용되었습니다. Johnson은 1981년 부통령직에서 물러난 뒤 부통령직을 수행했습니다. Johnson은 1991년에 부통령직을 수행했습니다. Johnson은 1971년 부통령직을 수행했습니다. Johnson은 1985년에 부통령직에 임명되었습니다. Johnson은 1982년에 부통령직을 수행했습니다. Johnson은 1986년에 부통령직을 수행했습니다. Johnson은 1993년에 부통

In [68]:
list_prompt = [
    # 원래 있던 것들
    '불고기용 고기 한우에요?',
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?',

    # 추가된 것들
    '한국에서 가장 높은 산은 어디야?',
    '서울 지하철 2호선은 몇 시에 끊겨?',
    'BTS 멤버 중 막내는 누구야?',
    '코로나19 첫 발생 연도는?',
    '한글날은 언제야?',
    '부산에서 유명한 음식은 뭐야?',
    '애플의 창립자는 누구야?',
    '인공지능과 머신러닝의 차이는 뭐야?',
    '한국의 전통 혼례에서 중요한 의식은?',
    '세계에서 가장 긴 강은 어디야?',
    '올해 한국 프로야구 우승팀은 누구야?',
    '김치찌개 맛있게 끓이는 법 알려줘',
    '삼국시대 고구려의 수도는 어디였어?',
    '테슬라 CEO는 누구야?',
    '아이 공부 집중력을 높이는 방법은?',
    '우주에서 가장 가까운 별 이름은 뭐야?'
]

In [69]:
list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text, actor)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 AI 언어 모델로 자연어를 이해해 드리고 있습니다. 일반적으로 불고기용 고기의 종류와 상황에 대한 정보를 알 수 없기 때문에, 질문의 정확한 답변을 제공할 수 없습니다. 정확한 내용을 원하신다면, 해당 식당이나 매장을 직접 방문하거나 SNS 계정을 통해 문의해주시기 바랍니다.\n\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n--\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n--\n-\n-\n-\n--\n-\n-\n-\n

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨이 47대 부통령직을 수행한 년도는 2021년이었습니다. J. E. 록펠러, J. W. 롤링스톤 등의 부통령이 역임하였습니다. J. E. 롤링스톤은 1963년 부통령직을 맡았습니다. J. E. 록펠러(사브론) 등이 재임 중입니다. W. 록펠러(사브론) 등이 부통령직을 맡았습니다. J. E. 록펠러, J. F. 롤링스톤, J. C. 롤링스톤 등이 있었습니다. J. E. 록펠러, J. E. 롤링스톤, J. C. 롤링스톤 등도 부통령직을 수행하였습니다. J. B. H. 록펠러, J. C. 롤링스톤, J. L. 록펠러 등이 역임하였습니다. J. W. 롤링스톤, J. L. 롤링스톤, L. L. 롤링스톤 등이 역임하였습니다. J. R. 록펠러, R. L. 롤링스톤, R. L. 롤링스톤 등의 부통령직이 있었습니다. J. E. 록펠러, L. 록펠러, L. W. 록펠러, J. W. 록펠러 등이 있었습니다. J. E. 록펠러 등 다른 부통령

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어

### Response(응답):'시카고 오헤어 국제공항은 주로 일본 도쿄 지